In [2]:
from pymilvus import MilvusClient, DataType
import numpy as np

In [3]:
client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

In [ ]:
data = [
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
    {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298"},
    {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794"},
    {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222"},
    {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392"},
    {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510"},
    {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381"},
    {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976"}
]

In [25]:
fields_to_add = [
    {
        'field_name': 'id',
        'datatype': DataType.INT64,
        'is_primary': True,
        'auto_id': False
    },
    {
        'field_name': 'vector',
        'datatype': DataType.FLOAT_VECTOR,
        'dim': 5
    },
    {
        'field_name': 'color',
        'datatype': DataType.VARCHAR,
        'max_length': 512
    }
]

In [ ]:
schema = MilvusClient.create_schema()

for field in fields_to_add:
    schema.add_field(
        collection_name="your_collection_name",
        **field  # 自动展开字典参数
    )
    print(f"已添加字段: {field['field_name']}")

已添加字段: id
已添加字段: vector
已添加字段: color


In [27]:
print(schema)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 5}}, {'name': 'color', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': False}


In [28]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="id",
    index_type="AUTOINDEX"
)

index_params.add_index(
    field_name="vector", 
    index_type="AUTOINDEX",
    metric_type="COSINE"
)

In [29]:
client.create_collection(
    collection_name="quick_setup",
    schema=schema,
    index_params=index_params
)

In [30]:
res = client.describe_collection(
    collection_name="quick_setup"
)

res

{'collection_name': 'quick_setup',
 'auto_id': False,
 'num_shards': 1,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'is_primary': True},
  {'field_id': 101,
   'name': 'vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 5}},
  {'field_id': 102,
   'name': 'color',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 512}}],
 'functions': [],
 'aliases': [],
 'collection_id': 463093537190374504,
 'consistency_level': 2,
 'properties': {'timezone': 'UTC'},
 'num_partitions': 1,
 'enable_dynamic_field': False,
 'created_timestamp': 463117423491940370,
 'update_timestamp': 463117423491940370}

In [31]:
res = client.insert(
    collection_name="quick_setup",
    # partition_name="partitionA",
    data=data
)

res

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'cost': 0}

In [33]:
# 基本 ANN 搜索
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]
res = client.search(
    collection_name="quick_setup",
    anns_field="vector",
    data=[query_vector],
    limit=3,
    search_params={"metric_type": "COSINE"}
)

for hits in res:
    for hit in hits:
        print(hit)

{'id': 0, 'distance': 1.0, 'entity': {}}
{'id': 1, 'distance': 0.6290165185928345, 'entity': {}}
{'id': 4, 'distance': 0.5975796580314636, 'entity': {}}


In [ ]:
# client.drop_collection(
#     collection_name="quick_setup"
# )

In [23]:
res = client.list_collections()

res

['demo_collection', 'customized_setup_1']